# 1. Installation des packages

<u>On installe ensuite à l'aide de la commande **pip** <br />
les packages qui nous seront nécessaires</u> :

In [1]:
!pip install Pandas pillow tensorflow pyspark pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=5aa206dafc59bff73ad92b4943a0e4f5457593bb7184c73180ebebec26f7519f
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


## 2. Import des librairies

In [2]:
import pandas as pd
from PIL import Image
import numpy as np
import io
import os

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession

In [18]:
from pyspark.ml.feature import PCA
from pyspark.ml.functions import array_to_vector
from pyspark.ml.functions import vector_to_array

## 3. Définition des PATH pour charger les images <br /> et enregistrer les résultats

Dans cette version locale nous partons du principe que les données <br />
sont stockées dans le même répertoire que le notebook.<br />
Nous n'utilisons qu'un extrait de **100 images** à traiter dans cette <br />
première version en local.<br />
L'extrait des images à charger est stockée dans le dossier **data**.<br />
Nous enregistrerons le résultat de notre traitement <br />
dans le dossier "**Results_Local**"

In [3]:
# lien vers google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
PATH = '/content/drive/MyDrive/Colab Notebooks/data'
PATH_Data = PATH
PATH_Result = '/content/drive/MyDrive/Colab Notebooks/results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

PATH:        /content/drive/MyDrive/Colab Notebooks/data
PATH_Data:   /content/drive/MyDrive/Colab Notebooks/data
PATH_Result: /content/drive/MyDrive/Colab Notebooks/results


## 4. Création de la SparkSession

L’application Spark est contrôlée grâce à un processus de pilotage (driver process) appelé **SparkSession**. <br />
<u>Une instance de **SparkSession** est la façon dont Spark exécute les fonctions définies par l’utilisateur <br />
dans l’ensemble du cluster</u>. <u>Une SparkSession correspond toujours à une application Spark</u>.

<u>Ici nous créons une session spark en spécifiant dans l'ordre</u> :
 1. un **nom pour l'application**, qui sera affichée dans l'interface utilisateur Web Spark "**P9**"
 2. que l'application doit s'exécuter **localement**. <br />
   Nous ne définissons pas le nombre de cœurs à utiliser (comme .master('local[4]) pour 4 cœurs à utiliser), <br />
   nous utiliserons donc tous les cœurs disponibles dans notre processeur.<br />
 3. une option de configuration supplémentaire permettant d'utiliser le **format "parquet"** <br />
   que nous utiliserons pour enregistrer et charger le résultat de notre travail.
 4. vouloir **obtenir une session spark** existante ou si aucune n'existe, en créer une nouvelle

In [5]:
spark = (SparkSession
             .builder
             .appName('P9')
             .master('local')
             .config("spark.sql.parquet.writeLegacyFormat", 'true')
             .getOrCreate()
)

<u>Nous créons également la variable "**sc**" qui est un **SparkContext** issue de la variable **spark**</u> :

In [6]:
sc = spark.sparkContext

<u>Affichage des informations de Spark en cours d'execution</u> :

In [7]:
spark

# 5. Traitement des données

<u>Dans la suite de notre flux de travail, <br />
nous allons successivement</u> :
1. Préparer nos données
    1. Importer les images dans un dataframe **pandas UDF**
    2. Associer aux images leur **label**
    3. Préprocesser en **redimensionnant nos images pour <br />
       qu'elles soient compatibles avec notre modèle**
2. Préparer notre modèle
    1. Importer le modèle **MobileNetV2**
    2. Créer un **nouveau modèle** dépourvu de la dernière couche de MobileNetV2
3. Définir le processus de chargement des images et l'application <br />
   de leur featurisation à travers l'utilisation de pandas UDF
3. Exécuter les actions d'extraction de features
4. Enregistrer le résultat de nos actions
5. Tester le bon fonctionnement en chargeant les données enregistrées




### 5.1 Chargement des données

Les images sont chargées au format binaire, ce qui offre, <br />
plus de souplesse dans la façon de prétraiter les images.

Avant de charger les images, nous spécifions que nous voulons charger <br />
uniquement les fichiers dont l'extension est **jpg**.

Nous indiquons également de charger tous les objets possibles contenus <br />
dans les sous-dossiers du dossier communiqué.

In [8]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

<u>Affichage des 5 premières images contenant</u> :
 - le path de l'image
 - la date et heure de sa dernière modification
 - sa longueur
 - son contenu encodé en valeur hexadécimal

<u>Je ne conserve que le **path** de l'image et j'ajoute <br />
    une colonne contenant les **labels** de chaque image</u> :

In [9]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+------------------------------------------------------------------+-----+
|path                                                              |label|
+------------------------------------------------------------------+-----+
|file:/content/drive/MyDrive/Colab Notebooks/data/Cocos/118_100.jpg|Cocos|
|file:/content/drive/MyDrive/Colab Notebooks/data/Cocos/168_100.jpg|Cocos|
|file:/content/drive/MyDrive/Colab Notebooks/data/Cocos/143_100.jpg|Cocos|
|file:/content/drive/MyDrive/Colab Notebooks/data/Cocos/93_100.jpg |Cocos|
|file:/content/drive/MyDrive/Colab Notebooks/data/Cocos/218_100.jpg|Cocos|
+------------------------------------------------------------------+-----+
only showing top 5 rows

None


### 5.2 Préparation du modèle

Je vais utiliser la technique du **transfert learning** pour extraire les features des images.<br />
J'ai choisi d'utiliser le modèle **MobileNetV2** pour sa rapidité d'exécution comparée <br />
à d'autres modèles comme *VGG16* par exemple.

Pour en savoir plus sur la conception et le fonctionnement de MobileNetV2, <br />
je vous invite à lire [cet article](https://towardsdatascience.com/review-mobilenetv2-light-weight-model-image-classification-8febb490e61c).

<u>Voici le schéma de son architecture globale</u> :

![Architecture de MobileNetV2](img/mobilenetv2_architecture.png)

Il existe une dernière couche qui sert à classer les images <br />
selon 1000 catégories que nous ne voulons pas utiliser.<br />
L'idée dans ce projet est de récupérer le **vecteur de caractéristiques <br />
de dimensions (1,1,1280)** qui servira, plus tard, au travers d'un moteur <br />
de classification à reconnaitre les différents fruits du jeu de données.

Comme d'autres modèles similaires, **MobileNetV2**, lorsqu'on l'utilise <br />
en incluant toutes ses couches, attend obligatoirement des images <br />
de dimension (224,224,3). Nos images étant toutes de dimension (100,100,3), <br />
nous devrons simplement les **redimensionner** avant de les confier au modèle.

<u>Dans l'odre</u> :
 1. Nous chargeons le modèle **MobileNetV2** avec les poids **précalculés** <br />
    issus d'**imagenet** et en spécifiant le format de nos images en entrée
 2. Nous créons un nouveau modèle avec:
  - <u>en entrée</u> : l'entrée du modèle MobileNetV2
  - <u>en sortie</u> : l'avant dernière couche du modèle MobileNetV2

In [10]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

14536120/14536120 [==============================] - 0s 0us/step


In [11]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

Affichage du résumé de notre nouveau modèle où nous constatons <br />
que <u>nous récupérons bien en sortie un vecteur de dimension (1, 1, 1280)</u> :

In [12]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

Tous les workeurs doivent pouvoir accéder au modèle ainsi qu'à ses poids. <br />
Une bonne pratique consiste à charger le modèle sur le driver puis à diffuser <br />
ensuite les poids aux différents workeurs.

In [13]:
brodcast_weights = sc.broadcast(new_model.get_weights())

<u>Mettons cela sous forme de fonction</u> :

In [14]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

### 5.3 Définition du processus de chargement des images et application <br/>de leur featurisation à travers l'utilisation de pandas UDF

Ce notebook définit la logique par étapes, jusqu'à Pandas UDF.

<u>L'empilement des appels est la suivante</u> :

- Pandas UDF
  - featuriser une série d'images pd.Series
   - prétraiter une image

In [15]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.

    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


### 5.4 Exécution des actions d'extraction de features

Les Pandas UDF, sur de grands enregistrements (par exemple, de très grandes images), <br />
peuvent rencontrer des erreurs de type Out Of Memory (OOM).<br />
Si vous rencontrez de telles erreurs dans la cellule ci-dessous, <br />
essayez de réduire la taille du lot Arrow via 'maxRecordsPerBatch'

Je n'utiliserai pas cette commande dans ce projet <br />
et je laisse donc la commande en commentaire.

In [16]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

Nous pouvons maintenant exécuter la featurisation sur l'ensemble de notre DataFrame Spark.<br />
<u>REMARQUE</u> : Cela peut prendre beaucoup de temps, tout dépend du volume de données à traiter. <br />

Notre jeu de données de **Test** contient **22819 images**. <br />
Cependant, dans l'exécution en mode **local**, <br />
nous <u>traiterons un ensemble réduit de **100 images**</u>.

In [19]:
features_df = images.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features"),
                                            featurize_udf("content").alias("features_vectors")).withColumn("features_vectors", array_to_vector("features_vectors"))

In [20]:
features_df.show(5, True)

+--------------------+------------+--------------------+--------------------+
|                path|       label|            features|    features_vectors|
+--------------------+------------+--------------------+--------------------+
|file:/content/dri...|   Carambula|[0.0017966543, 0....|[0.00179665430914...|
|file:/content/dri...|   Carambula|[0.063041806, 0.0...|[0.06304180622100...|
|file:/content/dri...|     Avocado|[0.45384344, 0.0,...|[0.45384344458580...|
|file:/content/dri...|Cactus fruit|[0.5492148, 0.088...|[0.54921478033065...|
|file:/content/dri...|  Grape Blue|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|
+--------------------+------------+--------------------+--------------------+
only showing top 5 rows



## 5.5 Application de la réduction de dimension

In [21]:
pca = PCA(k=100, inputCol = 'features_vectors', outputCol = 'pca_vectors')
model = pca.fit(features_df)
df_pca = model.transform(features_df)
df_pca.show(5, True)

+--------------------+------------+--------------------+--------------------+--------------------+
|                path|       label|            features|    features_vectors|         pca_vectors|
+--------------------+------------+--------------------+--------------------+--------------------+
|file:/content/dri...|   Carambula|[0.0017966543, 0....|[0.00179665430914...|[7.02984969477823...|
|file:/content/dri...|   Carambula|[0.063041806, 0.0...|[0.06304180622100...|[8.86773916807895...|
|file:/content/dri...|     Avocado|[0.45384344, 0.0,...|[0.45384344458580...|[-8.1079238361747...|
|file:/content/dri...|Cactus fruit|[0.5492148, 0.088...|[0.54921478033065...|[4.81757612642109...|
|file:/content/dri...|  Grape Blue|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|[-15.470275950760...|
+--------------------+------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [22]:
# ajout d'une colonne transformé vecteur ==> array
df_pca = df_pca.withColumn('pca_features', vector_to_array('pca_vectors'))
df_pca.show(5, True)

+--------------------+------------+--------------------+--------------------+--------------------+--------------------+
|                path|       label|            features|    features_vectors|         pca_vectors|        pca_features|
+--------------------+------------+--------------------+--------------------+--------------------+--------------------+
|file:/content/dri...|   Carambula|[0.0017966543, 0....|[0.00179665430914...|[7.02984969477823...|[7.02984969477823...|
|file:/content/dri...|   Carambula|[0.063041806, 0.0...|[0.06304180622100...|[8.86773916807895...|[8.86773916807895...|
|file:/content/dri...|     Avocado|[0.45384344, 0.0,...|[0.45384344458580...|[-8.1079238361747...|[-8.1079238361747...|
|file:/content/dri...|Cactus fruit|[0.5492148, 0.088...|[0.54921478033065...|[4.81757612642109...|[4.81757612642109...|
|file:/content/dri...|  Grape Blue|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|[-15.470275950760...|[-15.470275950760...|
+--------------------+------------+-----

<u>Rappel du PATH où seront inscrits les fichiers au format "**parquet**" <br />
contenant nos résultats, à savoir, un DataFrame contenant 3 colonnes</u> :
 1. Path des images
 2. Label de l'image
 3. Vecteur de caractéristiques de l'image
 4. Vecteur de caractéristiques de l'image au format vecteur
 5. Réduction de dimension l'image au format vecteur
 6. Réduction de dimension l'image

In [23]:
print(PATH_Result)

/content/drive/MyDrive/Colab Notebooks/results


<u>Enregistrement des données traitées au format "**parquet**"</u> :

In [24]:
df_pca.write.mode("overwrite").parquet(PATH_Result)

## 5.6 Chargement des données enregistrées et validation du résultat

<u>On charge les données fraichement enregistrées dans un **DataFrame Pandas**</u> :

In [25]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

<u>On affiche les 5 premières lignes du DataFrame</u> :

In [26]:
df.head()

,path,label,features,features_vectors,pca_vectors,pca_features
0,file:/content/drive/MyDrive/Colab Notebooks/da...,Carambula,"[0.0017966543, 0.0, 0.0, 0.0, 1.3657801, 0.0, ...","{'type': 1, 'size': None, 'indices': None, 'va...","{'type': 1, 'size': None, 'indices': None, 'va...","[7.029849694778237, -3.182887155471069, -5.842..."
1,file:/content/drive/MyDrive/Colab Notebooks/da...,Carambula,"[0.063041806, 0.0, 0.0, 0.0, 1.1863712, 0.0, 1...","{'type': 1, 'size': None, 'indices': None, 'va...","{'type': 1, 'size': None, 'indices': None, 'va...","[8.867739168078957, -3.090574602563129, -5.758..."
2,file:/content/drive/MyDrive/Colab Notebooks/da...,Avocado,"[0.45384344, 0.0, 0.011255186, 0.0, 0.01393795...","{'type': 1, 'size': None, 'indices': None, 'va...","{'type': 1, 'size': None, 'indices': None, 'va...","[-8.107923836174747, -7.7069649921312475, -5.2..."
3,file:/content/drive/MyDrive/Colab Notebooks/da...,Cactus fruit,"[0.5492148, 0.08829133, 0.0, 0.0, 0.06960537, ...","{'type': 1, 'size': None, 'indices': None, 'va...","{'type': 1, 'size': None, 'indices': None, 'va...","[4.817576126421097, -0.8011257986154534, 0.530..."
4,file:/content/drive/MyDrive/Colab Notebooks/da...,Grape Blue,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.375453, 0.05689919...","{'type': 1, 'size': None, 'indices': None, 'va...","{'type': 1, 'size': None, 'indices': None, 'va...","[-15.470275950760595, -1.6488918154359675, 1.9..."


<u>On valide que la dimension du vecteur de caractéristiques des images est bien de dimension 1280</u> :

In [27]:
df.loc[0,'features'].shape

(1280,)

<u>Sauvegarde du fichier en format csv<u> :

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/traitement_images.csv')